In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import scipy.ndimage

# Get data from CSV's

In [3]:
trainData = pd.read_csv("housing_train_clean.csv")
testData = pd.read_csv("housing_test_clean.csv")
trainData.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,True,False,False,False,False,False,...,False,True,8,856,True,False,0,2003,2003,2008
1,1262,0,0,3,True,False,False,False,False,False,...,False,True,6,1262,True,False,298,1976,1976,2007
2,920,866,0,3,True,False,False,False,False,False,...,False,True,6,920,True,False,0,2001,2002,2008
3,961,756,0,3,True,False,False,False,False,False,...,False,True,7,756,True,False,0,1915,1970,2006
4,1145,1053,0,4,True,False,False,False,False,False,...,False,True,9,1145,True,False,192,2000,2000,2008


In [4]:
testData.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,896,0,0,2,1,0,0,0,0,0,...,0,1,5,882.0,1,0,140,1961,1961,2010
1,1329,0,0,3,1,0,0,0,0,0,...,0,1,6,1329.0,1,0,393,1958,1958,2010
2,928,701,0,3,1,0,0,0,0,0,...,0,1,6,928.0,1,0,212,1997,1998,2010
3,926,678,0,3,1,0,0,0,0,0,...,0,1,7,926.0,1,0,360,1998,1998,2010
4,1280,0,0,2,0,0,0,0,1,0,...,0,1,5,1280.0,1,0,0,1992,1992,2010


In [5]:
trainData.shape, trainData.isnull().sum().sum(), testData.shape, testData.isnull().sum().sum()

((1460, 272), 0, (1459, 272), 0)

# Get Y_train and X_train

In [6]:
Y_train = trainData['SalePrice']
Y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [7]:
X_train = trainData.drop('SalePrice', axis=1)
X_train

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,True,False,False,False,False,False,...,False,True,8,856,True,False,0,2003,2003,2008
1,1262,0,0,3,True,False,False,False,False,False,...,False,True,6,1262,True,False,298,1976,1976,2007
2,920,866,0,3,True,False,False,False,False,False,...,False,True,6,920,True,False,0,2001,2002,2008
3,961,756,0,3,True,False,False,False,False,False,...,False,True,7,756,True,False,0,1915,1970,2006
4,1145,1053,0,4,True,False,False,False,False,False,...,False,True,9,1145,True,False,192,2000,2000,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,953,694,0,3,True,False,False,False,False,False,...,False,True,7,953,True,False,0,1999,2000,2007
1456,2073,0,0,3,True,False,False,False,False,False,...,False,True,7,1542,True,False,349,1978,1988,2010
1457,1188,1152,0,4,True,False,False,False,False,False,...,False,True,9,1152,True,False,0,1941,2006,2010
1458,1078,0,0,2,True,False,False,False,False,False,...,False,True,5,1078,True,False,366,1950,1996,2010


In [8]:
Y_train.shape, X_train.shape

((1460,), (1460, 271))

# Get X_test and Y_test

In [9]:
Y_test = testData['SalePrice']
Y_test

0       120376.994315
1       152387.635505
2       185025.193763
3       198385.651258
4       185934.885245
            ...      
1454     85890.984520
1455     83811.680275
1456    160189.586559
1457    115398.604894
1458    224009.963740
Name: SalePrice, Length: 1459, dtype: float64

In [10]:
X_test = testData.drop('SalePrice', axis=1)
X_test

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,896,0,0,2,1,0,0,0,0,0,...,0,1,5,882.0,1,0,140,1961,1961,2010
1,1329,0,0,3,1,0,0,0,0,0,...,0,1,6,1329.0,1,0,393,1958,1958,2010
2,928,701,0,3,1,0,0,0,0,0,...,0,1,6,928.0,1,0,212,1997,1998,2010
3,926,678,0,3,1,0,0,0,0,0,...,0,1,7,926.0,1,0,360,1998,1998,2010
4,1280,0,0,2,0,0,0,0,1,0,...,0,1,5,1280.0,1,0,0,1992,1992,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,546,546,0,3,0,0,0,1,0,0,...,0,1,5,546.0,1,0,0,1970,1970,2006
1455,546,546,0,3,0,0,0,0,1,0,...,0,1,6,546.0,1,0,0,1970,1970,2006
1456,1224,0,0,4,1,0,0,0,0,0,...,0,1,7,1224.0,1,0,474,1960,1996,2006
1457,970,0,0,3,1,0,0,0,0,0,...,0,1,6,912.0,1,0,80,1992,1992,2006


In [11]:
Y_test.shape, X_test.shape

((1459,), (1459, 271))

# Convert pandas datatype to Numpy array

In [12]:
X_train = X_train.values
Y_train = Y_train.values.reshape(-1)

X_test = X_test.values
Y_test = Y_test.values.reshape(-1)

# Normalize X_train and X_test using MinMaxScaler

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train, X_train.shape, Y_train.shape

(array([[0.11977972, 0.41355932, 0.        , ..., 0.94927536, 0.88333333,
         0.5       ],
        [0.21294172, 0.        , 0.        , ..., 0.75362319, 0.43333333,
         0.25      ],
        [0.13446535, 0.41937046, 0.        , ..., 0.93478261, 0.86666667,
         0.5       ],
        ...,
        [0.19596145, 0.55786925, 0.        , ..., 0.5       , 0.93333333,
         1.        ],
        [0.17072051, 0.        , 0.        , ..., 0.56521739, 0.76666667,
         1.        ],
        [0.21156494, 0.        , 0.        , ..., 0.67391304, 0.25      ,
         0.5       ]]),
 (1460, 271),
 (1460,))

In [15]:
X_test, X_test.shape, Y_test.shape

(array([[0.12895824, 0.        , 0.        , ..., 0.64492754, 0.18333333,
         1.        ],
        [0.22831574, 0.        , 0.        , ..., 0.62318841, 0.13333333,
         1.        ],
        [0.13630106, 0.33946731, 0.        , ..., 0.9057971 , 0.8       ,
         1.        ],
        ...,
        [0.20422212, 0.        , 0.        , ..., 0.63768116, 0.76666667,
         0.        ],
        [0.1459385 , 0.        , 0.        , ..., 0.86956522, 0.7       ,
         0.        ],
        [0.15190454, 0.48619855, 0.        , ..., 0.87681159, 0.73333333,
         0.        ]]),
 (1459, 271),
 (1459,))

# Get the validation set

In [16]:
pureTrain = int(0.9 * X_train.shape[0]) # get 90% of the training data to be for "pure" training
pureTrain

1314

In [17]:
# Get the indexes of all the training samples, shuffle them around
# then pick the first 90% for training and remaining 10% for validation
indexList = np.arange(0, X_train.shape[0])
np.random.shuffle(indexList)
trainIndex = indexList[0:pureTrain]
valIndex = indexList[pureTrain:]

# KNeighborsRegressor

In [18]:
nNeighbors = np.arange(1, 100, 10)
paramGridKNN = {'n_neighbors':nNeighbors}

In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
KNNmodel = KNeighborsRegressor()
gridSearchKNN = GridSearchCV(estimator=KNNmodel, param_grid=paramGridKNN, scoring='neg_mean_squared_error', cv=5)

In [20]:
gridSearchKNN.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             scoring='neg_mean_squared_error')

In [21]:
MSE_validation_KNN = -gridSearchKNN.cv_results_['mean_test_score'] # 'test' is validation
MSE_validation_KNN

array([2.71917262e+09, 1.86028731e+09, 1.97309487e+09, 2.07423042e+09,
       2.13711668e+09, 2.23161410e+09, 2.32169924e+09, 2.39379685e+09,
       2.46074223e+09, 2.51740690e+09])

In [22]:
gridSearchKNN.best_params_

{'n_neighbors': 11}

In [23]:
bestKNN = gridSearchKNN.best_estimator_
bestKNN

KNeighborsRegressor(n_neighbors=11)

In [24]:
Y_train_pred_KNN = bestKNN.predict(X_train)
MSE_train_KNN = np.mean((Y_train_pred_KNN - Y_train)**2)
MSE_train_KNN

1450341915.4822032

In [25]:
Y_test_pred_KNN = bestKNN.predict(X_test)

In [26]:
MSE_test_KNN = np.mean((Y_test_pred_KNN - Y_test)**2)
MSE_test_KNN

1377009915.4748838

In [27]:
MAE_test_KNN = np.mean(np.abs(Y_test - Y_test_pred_KNN))
MAE_test_KNN

22448.206520922788

In [28]:

MAPE_test_KNN = np.mean(np.abs((Y_test - Y_test_pred_KNN) / Y_test))
MAPE_test_KNN

0.12691424024610484

# Decision Tree Regressor

In [29]:
maxDepthDT = np.arange(1, 100, 10)
paramGridDT = {'max_depth':maxDepthDT}

In [30]:
from sklearn.tree import DecisionTreeRegressor
DTmodel = DecisionTreeRegressor()
gridSearchDT = GridSearchCV(estimator=DTmodel, param_grid=paramGridDT, scoring='neg_mean_squared_error', cv=5)

In [31]:
gridSearchDT.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             scoring='neg_mean_squared_error')

In [32]:
MSE_validation_DT = -gridSearchDT.cv_results_['mean_test_score']
MSE_validation_DT

array([3.58726852e+09, 1.51573908e+09, 1.57193171e+09, 1.48821153e+09,
       1.77348932e+09, 1.58764723e+09, 1.74686285e+09, 1.70163875e+09,
       1.72586908e+09, 1.59942194e+09])

In [33]:
gridSearchDT.best_params_

{'max_depth': 31}

In [34]:
bestDT = gridSearchDT.best_estimator_
bestDT

DecisionTreeRegressor(max_depth=31)

In [35]:
Y_train_pred_DT = bestDT.predict(X_train)
MSE_train_DT = np.mean((Y_train_pred_DT - Y_train)**2)
MSE_train_DT

0.0

In [36]:
Y_test_pred_DT = bestDT.predict(X_test)

In [37]:
MSE_test_DT = np.mean((Y_test_pred_DT - Y_test)**2)
MSE_test_DT

1613338451.8982413

In [38]:
MAE_test_DT = np.mean(np.abs(Y_test - Y_test_pred_DT))
MAE_test_DT

22014.641667041065

In [39]:
MAPE_test_DT = np.mean(np.abs((Y_test - Y_test_pred_DT) / Y_test))
MAPE_test_DT

0.11942112926627835

# Random Forest Regressor

In [40]:
maxDepthRF = np.arange(1, 100, 10)
paramGridRF = {'max_depth':maxDepthRF}

In [41]:
from sklearn.ensemble import RandomForestRegressor
RFmodel = RandomForestRegressor()
gridSearchRF = GridSearchCV(estimator=RFmodel, param_grid=paramGridRF, scoring='neg_mean_squared_error', cv=5)

In [42]:
gridSearchRF.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             scoring='neg_mean_squared_error')

In [43]:
MSE_validation_RF = -gridSearchRF.cv_results_['mean_test_score']
MSE_validation_RF

array([3.12497999e+09, 9.25331246e+08, 9.12041474e+08, 9.37944424e+08,
       9.28743467e+08, 9.15547661e+08, 9.21902482e+08, 8.99436817e+08,
       8.98611487e+08, 9.19289352e+08])

In [44]:
gridSearchRF.best_params_

{'max_depth': 81}

In [45]:
bestRF = gridSearchRF.best_estimator_
bestRF

RandomForestRegressor(max_depth=81)

In [46]:
Y_train_pred_RF = bestRF.predict(X_train)
MSE_train_RF = np.mean((Y_train_pred_RF - Y_train)**2)
MSE_train_RF

132304892.29888315

In [47]:
Y_test_pred_RF = bestRF.predict(X_test)

In [48]:
MSE_test_RF = np.mean((Y_test_pred_RF - Y_test)**2)
MSE_test_RF

552371088.520145

In [49]:
MAE_test_RF = np.mean(np.abs(Y_test - Y_test_pred_RF))
MAE_test_RF

10750.22380093818

In [50]:
MAPE_test_RF = np.mean(np.abs((Y_test - Y_test_pred_RF) / Y_test))
MAPE_test_RF

0.061653462169563904

In [51]:
MSE = [MSE_test_KNN, MSE_test_DT, MSE_test_RF]
MAE = [MAE_test_KNN, MAE_test_DT, MAE_test_RF]
MAPE = [MAPE_test_KNN, MAPE_test_DT, MAPE_test_RF]
rows = ["MSE", "MAE", "MAPE"]
cols = ["KNeighbors Regressor", "DecisionTree Regressor", "RandomForest Regressor"]

In [52]:
results = pd.DataFrame((MSE, MAE, MAPE), index=rows, columns=cols)
results.head()

,KNeighbors Regressor,DecisionTree Regressor,RandomForest Regressor
MSE,1.377010e+09,1.613338e+09,5.523711e+08
MAE,2.244821e+04,2.201464e+04,1.075022e+04
MAPE,1.269142e-01,1.194211e-01,6.165346e-02
